In [49]:
import math
import string

tests = """
3:20
"""

class Range:
    def __init__(self, start, stop, step=1):
        self.start = start
        self.stop = stop
        self.step = step
        
    def evaluate(self):
        return list(range(self.start, self.stop+1, self.step))

class Token:
    def __init__(self, string='', token_type=None):
        self.string = string
        self.token_type = token_type

class Array:
    def __init__(self, terms):
        self.data = []
        for t in terms:
            if type(t) is list:
                self.data.append(array(t))
            else:
                if t not in list(',;'):
                  self.data.append(t)

class Program:
    def __init__(self, source):
        self.source = source
        self.tree = []
        
        self.char_sets = {
            'op': '!@#$%^&*/-+<>',
            'syntax': '()[]{},.;:=|',
            'letter': string.ascii_lowercase,
            'numeric': string.digits + '.'
        }
        
        self.parse()
        
    def char_type(self, x):
#         return list(filter(lambda x: k for k, v in self.char_sets if x in v))[0]
        return [k for k, v in self.char_sets.items() if x in v][0]
        
    def parse(self):
        statements = self.source.replace('|', '\n').split('\n')
        statements = list(filter(None, statements))
        for s in statements:
            statement_parse = []
            token = Token(s[0], self.char_type(s[0]))
            for c in s[1:]:
                c_type = self.char_type(c)
                if c_type == token.token_type:
                    token.string += c
                else:
                    statement_parse.append(token)
                    token = Token(c, c_type)
            statement_parse.append(token)
            self.tree.append(statement_parse)
        
M = Program(tests)
M.tree[0]